In [0]:
# Data Science com o Registro de Ocupação Hospitalar COVID-19
# https://opendatasus.saude.gov.br/dataset/registro-de-ocupacao-hospitalar

import requests
import json
import time
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import ArrayType, StructType, DateType, TimestampType
from pyspark.sql.functions import max, col, size, array_contains, map_keys, explode, explode_outer, to_timestamp

def normaliza(nested_df):
  # Script por Amit Rawat
  # https://intellipaat.com/community/13569/how-to-flatten-a-struct-in-a-spark-dataframe
  
  flat_cols = [c[0] for c in nested_df.dtypes if c[1][:6] != 'struct']
  nested_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']
  flat_df = nested_df.select(flat_cols +
                             [col(nc+'.'+c).alias(nc+'_'+c)
                              for nc in nested_cols
                              for c in nested_df.select(nc+'.*').columns]
                            )
  return flat_df

# Credências de acesso da API

usuario = "user-api-leitos"
senha = "aQbLL3ZStaTr38tj"

# Aquisição da base de dados

url = "https://elastic-leitos.saude.gov.br/leito_ocupacao/_search"
r = requests.post(
  url, 
  auth=(usuario, senha), 
  json={
    "size": 10000, 
    "query": {"match_all": {}}
  }
)

base = r.json()["hits"]["hits"]

f = []
for i in base:
  f.append(json.dumps(i))

base = spark.read.json(sc.parallelize(f))

# Pré processamento

base = normaliza(base) # Normaliza colunas
base = base.withColumn("_source_dataNotificacaoOcupacao", col('_source_dataNotificacaoOcupacao').cast(TimestampType())) # Corrige timestamp da data de notificação da ocupação

base.printSchema()
print(base.count())

root
-- _id: string (nullable = true)
-- _index: string (nullable = true)
-- _score: double (nullable = true)
-- _type: string (nullable = true)
-- _source_algumaOcupacaoInformada: boolean (nullable = true)
-- _source_altas: double (nullable = true)
-- _source_cnes: string (nullable = true)
-- _source_dataNotificacaoOcupacao: timestamp (nullable = true)
-- _source_estado: string (nullable = true)
-- _source_estadoSigla: string (nullable = true)
-- _source_municipio: string (nullable = true)
-- _source_nomeCnes: string (nullable = true)
-- _source_obitos: long (nullable = true)
-- _source_ocupHospCli: double (nullable = true)
-- _source_ocupHospUti: double (nullable = true)
-- _source_ocupSRAGCli: double (nullable = true)
-- _source_ocupSRAGUti: long (nullable = true)
-- _source_ocupacaoInformada: boolean (nullable = true)
-- _source_ofertaHospCli: long (nullable = true)
-- _source_ofertaHospUti: long (nullable = true)
-- _source_ofertaRespiradores: long (nullable = true)
-- _source_ofertaSRAGCli: long (nullable = true)
-- _source_ofertaSRAGUti: long (nullable = true)

3936

In [0]:
### Qual a contagem de leitos ocupados por estado?

base \
  .groupBy(col("_source_estado")) \
  .count() \
  .select(col("_source_estado").alias("Estado"), col("count").alias("Leitos")) \
  .sort(col("Leitos").desc()) \
  .show(truncate=False)

+-------------------+------+
Estado |Leitos|
+-------------------+------+
São Paulo |747 |
Minas Gerais |463 |
Goiás |417 |
Rio Grande do Sul |316 |
Ceará |263 |
Rio de Janeiro |208 |
Pernambuco |208 |
Paraná |184 |
Mato Grosso |167 |
Bahia |150 |
Santa Catarina |124 |
Mato Grosso do Sul |116 |
Pará |72 |
Amazonas |69 |
Maranhão |68 |
Rio Grande do Norte|67 |
Espírito Santo |49 |
Piauí |49 |
Tocantins |35 |
Rondônia |32 |
+-------------------+------+
only showing top 20 rows

In [0]:
### Qual a contagem de leitos ocupados por cidade?

base \
  .filter(col("_source_nomeCnes").isNotNull()) \
  .groupBy(col("_source_nomeCnes")) \
  .count() \
  .select(col("_source_nomeCnes").alias("Hospital"), col("count").alias("Leitos")) \
  .sort(col("Leitos").desc()) \
  .show(truncate=False)

+-----------------------------------------------+------+
Hospital |Leitos|
+-----------------------------------------------+------+
HOSPITAL DE CAMPANHA COVID 19 |19 |
HOSPITAL SAO JOSE |18 |
HOSPITAL SAO VICENTE DE PAULO |15 |
HOSPITAL SAO LUCAS |13 |
HOSPITAL SAO FRANCISCO |12 |
HOSPITAL SANTO ANTONIO |12 |
HOSPITAL UNIMED |8 |
HOSPITAL SAO LUIZ |7 |
HOSPITAL NOSSA SENHORA APARECIDA |6 |
HOSPITAL NOSSA SENHORA DAS GRACAS |6 |
HOSPITAL BOM PASTOR |6 |
HOSPITAL SANTA ISABEL |6 |
HOSPITAL E MATERNIDADE NOSSA SENHORA DAS GRACAS|5 |
SANTA CASA |5 |
HOSPITAL SAO SEBASTIAO |5 |
HOSPITAL DO CORACAO |5 |
HOSPITAL BOM JESUS |4 |
HOSPITAL NOSSA SENHORA DAS DORES |4 |
HOSPITAL SAO CAMILO |4 |
HOSPITAL MUNICIPAL SAO SEBASTIAO |4 |
+-----------------------------------------------+------+
only showing top 20 rows

In [0]:
### Qual a contagem de leitos ocupados por hospital?

base \
  .groupBy(col("_source_municipio")) \
  .count() \
  .select(col("_source_municipio").alias("Cidade"), col("count").alias("Leitos")) \
  .sort(col("Leitos").desc()) \
  .show(truncate=False)

+-------------------+------+
Cidade |Leitos|
+-------------------+------+
São Paulo |111 |
Goiânia |88 |
Rio de Janeiro |76 |
Fortaleza |55 |
Manaus |36 |
Curitiba |35 |
Salvador |35 |
Recife |35 |
Belo Horizonte |29 |
Natal |27 |
Porto Alegre |23 |
Campo Grande |22 |
Brasília |21 |
Cuiabá |20 |
Campinas |17 |
São Luís |16 |
Teresina |15 |
São José dos Campos|15 |
Belém |13 |
Florianópolis |13 |
+-------------------+------+
only showing top 20 rows

In [0]:
### Quais os hospitais com leitos ocupados em Rolândia - PR

base \
  .filter(col("_source_municipio") == "Rolândia") \
  .groupBy(col("_source_nomeCnes")) \
  .count() \
  .select(col("_source_nomeCnes").alias("Hospital"), col("count").alias("Leitos")) \
  .show(truncate=False)

+-------------------+------+
Hospital |Leitos|
+-------------------+------+
HOSPITAL SAO RAFAEL|1 |
+-------------------+------+